In [1]:
from brainlit.utils import read_swc, df_to_graph, graph_to_paths
from brainlit.viz.visualize import napari_viewer
import numpy as np
from skimage import io
from scipy.ndimage.morphology import distance_transform_edt
from pathlib import Path
from brainlit.algorithms.image_processing import Bresenham3D
from brainlit.utils.benchmarking_params import brain_offsets, vol_offsets, scales, type_to_date

D:\Anaconda3\envs\brainlit\lib\site-packages\napari\__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
# loading all the benchmarking images from local paths
# all the paths of gfp images are saved in variable gfp_files
# the folder of output masks is in the same folder where the folder of benchmarking data is
base_dir = Path("D:/Study/Nuero Data Design/brainlit")
data_dir = base_dir / "benchmarking_datasets"
im_dir = data_dir / "Images"
mask_dir = base_dir / "benchmarking_masks"
gfp_files = list(im_dir.glob("**/*.tif"))
swc_base_path = data_dir / "Manual-GT"
save = True

for im_num, im_path in enumerate(gfp_files):
    # loading one gfp image
    print(str(im_path))
    im = io.imread(im_path, plugin="tifffile")
    im = np.swapaxes(im, 0, 2)
    
    file_name = im_path.parts[-1][:-8]
    
    f = im_path.parts[-1][:-8].split("_")
    image = f[0]
    date = type_to_date[image]
    num = int(f[1])

    scale = scales[date]
    brain_offset = brain_offsets[date]
    vol_offset = vol_offsets[date][num]
    im_offset = np.add(brain_offset, vol_offset)

    # loading all the .swc files corresponding to the image
    # all the paths of .swc files are saved in variable swc_files
    lower = int(np.floor((num - 1) / 5) * 5 + 1)
    upper = int(np.floor((num - 1) / 5) * 5 + 5)
    dir1 = date + "_" + image + "_" + str(lower) + "-" + str(upper)
    dir2 = date + "_" + image + "_" + str(num)
    swc_path = swc_base_path / dir1 / dir2
    swc_files = list(swc_path.glob("**/*.swc"))

    paths_total = []
    labels_total = np.zeros(im.shape)
    
    # generate paths and save them into paths_total
    for swc_num, swc in enumerate(swc_files):
        if "cube" in swc.parts[-1]:
            # skip the bounding box swc
            continue
        print(swc)
        df, swc_offset, _, _, _ = read_swc(swc)

        offset_diff = np.subtract(swc_offset, im_offset)
        G = df_to_graph(df)

        paths = graph_to_paths(G)

        # for every path in that swc
        for path_num, p in enumerate(paths):
            pvox = (p + offset_diff) / (scale) * 1000
            paths_total.append(pvox)
    
    # generate labels by using paths
    for path_voxel in paths_total:
        for voxel_num, voxel in enumerate(path_voxel):
            if voxel_num == 0:
                continue
            voxel_prev = path_voxel[voxel_num-1,:]
            xs,ys,zs = Bresenham3D(int(voxel_prev[0]), int(voxel_prev[1]), int(voxel_prev[2]),int(voxel[0]), int(voxel[1]), int(voxel[2]))
            for x,y,z in zip(xs,ys,zs):
                vox = np.array((x,y,z))
                if (vox >= 0).all() and (vox < im.shape).all():
                    labels_total[x,y,z] = 1  
    
    label_flipped = labels_total*0
    label_flipped[labels_total==0] = 1
    dists = distance_transform_edt(label_flipped, sampling = scale)
    labels_total[dists <= 1000] = 1
    
    if save:
        im_file_name = file_name + "_mask.tif"
        out_file = mask_dir / im_file_name
        io.imsave(out_file, labels_total, plugin="tifffile")

D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\test_1-gfp.tif
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_1.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_10.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_11.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_12.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_13.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_14.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_15.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_1-5\8-01_test_1\tree_16.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_

D:\Anaconda3\envs\brainlit\lib\site-packages\ipykernel_launcher.py:80: UserWarning: D:\Study\Nuero Data Design\brainlit\benchmarking_masks\test_1_mask.tif is a low contrast image


D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\test_10-gfp.tif
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_1.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_2.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_3.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_4.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_5.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_6.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_7.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_6-10\8-01_test_10\tree_8.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manua

D:\Anaconda3\envs\brainlit\lib\site-packages\ipykernel_launcher.py:80: UserWarning: D:\Study\Nuero Data Design\brainlit\benchmarking_masks\test_10_mask.tif is a low contrast image


D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\test_11-gfp.tif
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_11-15\8-01_test_11\tree_1.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_11-15\8-01_test_11\tree_2.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_11-15\8-01_test_11\tree_3.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\8-01_test_11-15\8-01_test_11\tree_4.swc


...

D:\Anaconda3\envs\brainlit\lib\site-packages\ipykernel_launcher.py:80: UserWarning: D:\Study\Nuero Data Design\brainlit\benchmarking_masks\validation_7_mask.tif is a low contrast image


D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\validation_8-gfp.tif
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_1.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_10.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_11.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_12.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_13.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_14.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_8\tree_15.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\

D:\Anaconda3\envs\brainlit\lib\site-packages\ipykernel_launcher.py:80: UserWarning: D:\Study\Nuero Data Design\brainlit\benchmarking_masks\validation_8_mask.tif is a low contrast image


D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\validation_9-gfp.tif
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_1.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_10.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_11.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_12.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_13.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_14.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\10-01_validation_6-10\10-01_validation_9\tree_15.swc
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Manual-GT\

D:\Anaconda3\envs\brainlit\lib\site-packages\ipykernel_launcher.py:80: UserWarning: D:\Study\Nuero Data Design\brainlit\benchmarking_masks\validation_9_mask.tif is a low contrast image


In [3]:
# checking whether masks can be loaded
show_napari = False
mask_files = list(mask_dir.glob("**/*.tif"))

for im_num, im_path in enumerate(gfp_files):
    im = io.imread(im_path, plugin="tifffile")
    im = np.swapaxes(im, 0, 2)
    
    file_name = im_path.parts[-1][:-8]
    mask_file = file_name + "_mask.tif"
    mask_path = mask_dir / mask_file
    mask = io.imread(mask_path, plugin="tifffile")
    
    print("loading the mask of", file_name, "...")
    if show_napari:
        napari_viewer(im, labels=mask, label_name="mask")

loading the mask of test_1 ...
loading the mask of test_10 ...
loading the mask of test_11 ...
loading the mask of test_12 ...
loading the mask of test_13 ...
loading the mask of test_14 ...
loading the mask of test_15 ...
loading the mask of test_16 ...
loading the mask of test_17 ...
loading the mask of test_18 ...
loading the mask of test_19 ...
loading the mask of test_2 ...
loading the mask of test_20 ...
loading the mask of test_21 ...
loading the mask of test_22 ...
loading the mask of test_23 ...
loading the mask of test_24 ...
loading the mask of test_25 ...
loading the mask of test_3 ...
loading the mask of test_4 ...
loading the mask of test_5 ...
loading the mask of test_6 ...
loading the mask of test_7 ...
loading the mask of test_8 ...
loading the mask of test_9 ...
loading the mask of validation_1 ...
loading the mask of validation_10 ...
loading the mask of validation_11 ...
loading the mask of validation_12 ...
loading the mask of validation_13 ...
loading the mask of 